### Run this in Python kernel

In [2]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add_aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio

In [3]:
;; initialise with current date and initial capital (= $10000)
(init_portfolio "1980-12-16" 10000);

### Write a strategy

The following code implements a simple trading strategy:

In a timespan of 10 days (inclusive of today),
- Buy 50 stocks of AAPL on the first day
- Sell 10 stocks of AAPL on every other day

In [4]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 10))                              

(while (pos? @num-of-days)
    (do 
        ;; write your trading strategy here
        (if (= 10 @num-of-days)
            (do
                (order "AAPL" 50) ; buy 50 stocks
                (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get_date)))
            )
        )
        (if (odd? @num-of-days)
            (do
                (order "AAPL" -10) ; sell 10 stocks
                (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get_date)))
            )
        )
        
        (update-eval-report (get_date))
        
        ; move on to the next trading day
        (next_date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

; check whether counter == 0
(println ((fn [counter] (str "Counter: " counter)) @num-of-days))

26.093628318359524
Buy 50 stocks of AAPL on 1980-12-16
1126.4186732315392
Sell 10 stocks of AAPL on 1980-12-17
111127.669667928008312
Sell 10 stocks of AAPL on 1980-12-19
555528.803032950478386
Sell 10 stocks of AAPL on 1980-12-23
777730.10884791386528
Sell 10 stocks of AAPL on 1980-12-26
1212121229.41870999204002
Sell 10 stocks of AAPL on 1980-12-30
1414Counter: 0


nil

### Check order record

In [5]:
(pprint/print-table (deref order_record))


|      :date | :tic |  :price | :quantity | :total | :reference |
|------------+------+---------+-----------+--------+------------|
| 1980-12-17 | AAPL | 25.9375 |        50 |     50 |          2 |
| 1980-12-18 | AAPL | 26.6875 |       -10 |     40 |          3 |
| 1980-12-22 | AAPL | 29.6875 |       -10 |     30 |          5 |
| 1980-12-24 | AAPL | 32.5625 |       -10 |     20 |          7 |
| 1980-12-29 | AAPL | 36.0625 |       -10 |     10 |          9 |
| 1980-12-31 | AAPL | 34.1875 |       -10 |      0 |         11 |


nil

### Check portfolio record

In [6]:
;; view final portfolio
(view_portfolio)


| :asset |  :price | :aprc | :quantity | :tot_val |
|--------+---------+-------+-----------+----------|
|   cash |     N/A |   N/A |       N/A |    10295 |
|   AAPL | 34.1875 | 29.42 |         0 |        0 |


nil

In [7]:
;; view portfolio value and return
(view_portfolio_record)


|      :date | :tot_value | :daily_ret |
|------------+------------+------------|
| 1980-12-16 |     $10000 |      0.00% |
| 1980-12-17 |     $10007 |      0.00% |
| 1980-12-18 |     $10026 |      0.00% |
| 1980-12-22 |     $10096 |      0.01% |
| 1980-12-24 |     $10168 |      0.01% |
| 1980-12-29 |     $10254 |      0.01% |
| 1980-12-31 |     $10295 |      0.00% |


nil

### Generate evaluation report

In [8]:
(eval-report)


|      :date | :pnl-pt | :ret-da | :ret-r | :ret-tot | :sharpe-e | :sharpe-r | :tot-val | :vol-e | :vol-r |
|------------+---------+---------+--------+----------+-----------+-----------+----------+--------+--------|
| 1980-12-16 |   $7.81 |   0.08% |  0.22% |    0.00% |     1.41% |    24.80% |   $10007 |  0.06% |  0.88% |
| 1980-12-17 |  $13.37 |   0.19% |  0.96% |    0.00% |     2.81% |    63.55% |   $10026 |  0.10% |  1.51% |
| 1980-12-18 |  $13.37 |   0.19% |  0.96% |    0.00% |     2.81% |    63.55% |   $10026 |  0.10% |  1.51% |
| 1980-12-19 |  $32.32 |   0.70% |  0.62% |    0.01% |     3.07% |    12.48% |   $10096 |  0.31% |  4.99% |
| 1980-12-22 |  $32.32 |   0.70% |  0.62% |    0.01% |     3.07% |    12.48% |   $10096 |  0.31% |  4.99% |
| 1980-12-23 |  $42.14 |   0.71% |  0.82% |    0.02% |     4.88% |    15.01% |   $10168 |  0.34% |  5.44% |
| 1980-12-24 |  $42.14 |   0.71% |  0.82% |    0.02% |     4.88% |    15.01% |   $10168 |  0.34% |  5.44% |
| 1980-12-26 |  $50.84 |   

nil

### Plot variables

In [ ]:
;; to-do